In [87]:
import os, sys
sys.path.insert(1, os.path.abspath("../../../"))

In [88]:
import numpy as np
import pandas as pd
import logging

import dowhy
from dowhy import CausalModel
import dowhy.datasets

In [101]:
import econml
from econml.dml import DMLCateEstimator, SparseLinearDMLCateEstimator
from sklearn.preprocessing import PolynomialFeatures
from econml.bootstrap import BootstrapEstimator
from sklearn.linear_model import MultiTaskElasticNet

import numpy as np

# Instance parameters
n_controls = 2
n_instruments = 1
n_features = 1
n_treatments = 1
alpha = np.random.normal(size=(n_controls, 1))
beta = np.random.normal(size=(n_instruments, 1))
gamma = np.random.normal(size=(n_treatments, 1))
delta = np.random.normal(size=(n_treatments, 1))
zeta = np.random.normal(size=(n_controls, 1))

n_samples = 10000
W = np.random.normal(size=(n_samples, n_controls))
Z = np.random.normal(size=(n_samples, n_instruments))
X = np.random.normal(size=(n_samples, n_features))
eta = np.random.normal(size=(n_samples, n_treatments))
epsilon = np.random.normal(size=(n_samples, 1))
T = np.dot(W, alpha) + np.dot(Z, beta) + eta
y = np.dot(T**2, gamma) + np.dot(np.multiply(T, X), delta) + np.dot(W, zeta) + epsilon
print(gamma, delta)

[[-1.06726503]] [[-0.21029007]]


In [102]:
# Fit counterfactual model
cfest = DMLCateEstimator(model_y=MultiTaskElasticNet(alpha=0.1),
                    model_t=MultiTaskElasticNet(alpha=0.1),
                    featurizer=PolynomialFeatures(degree=1, include_bias=False))
cfest.fit(y, T, X, W)

In [103]:
X_test = X
# Estimate heterogeneous treatment effects from going from treatment 0 to treatment 1
T0_test = np.zeros((X_test.shape[0], n_treatments))
T1_test = np.ones((X_test.shape[0], n_treatments))
#hetero_te = cfest.effect(T0_test, T1_test, X_test)
te_pred = est.const_marginal_effect(np.array([[np.median(X)]]))
print(te_pred)

[0.08682089]


In [110]:
import sklearn.ensemble
from sklearn.linear_model import ElasticNetCV

#est = DMLCateEstimator(model_y=sklearn.ensemble.RandomForestRegressor(),
#                       model_t=sklearn.ensemble.RandomForestRegressor(),
#                        featurizer=sklearn.preprocessing.PolynomialFeatures(degree=5))
est = SparseLinearDMLCateEstimator()
est.fit(np.ravel(y), np.ravel(T), X, W)
X_test = X
# Estimate heterogeneous treatment effects from going from treatment 0 to treatment 1
T0_test = np.zeros((X_test.shape[0], n_treatments))
T1_test = np.ones((X_test.shape[0], n_treatments))
print(est.effect(X, np.ravel(T0_test), np.ravel(T1_test))[:5])
# To get the coefficients of the polynomial fitted in the final stage we can
# access the coef_ attribute of the fitted second stage model. This would
# return the coefficients in front of each term in the vector T⊗ϕ(X).
a_hat = est.coef_
y0 = np.dot(T0_test, gamma) + np.dot(np.multiply(T0_test, X), delta) + np.dot(W, zeta) + epsilon
y1 = np.dot(T1_test, gamma) + np.dot(np.multiply(T1_test, X), delta) + np.dot(W, zeta) + epsilon
tau = y1-y0
print(tau[:5])
print(a_hat) 
print(gamma, delta)

/home/amit/.local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/amit/.local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/amit/.local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/home/amit/.local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_

[ 0.09582626  0.22194015  0.31151788 -0.00683608  0.29379471]
[[-0.98257543]
 [-0.82766611]
 [-0.7176352 ]
 [-1.10867854]
 [-0.73940508]]
[ 0.02687925 -0.17120015]
[[-1.06726503]] [[-0.21029007]]


In [96]:
# Estimate heterogeneous marginal effects around treatment 0
T_test = np.zeros((X_test.shape[0], n_treatments))
hetero_marginal_te = est.marginal_effect(T_test, X_test)
print(y[1])
hetero_marginal_te[1]

[-5.13365983]


In [64]:
# Estimate average treatment effects over a population of z's
T0_test = np.zeros((X_test.shape[0], n_treatments))
T1_test = np.ones((X_test.shape[0], n_treatments))

# average treatment effect
ate = np.mean(est.effect(T0_test, T1_test, X_test)) # returns estimate of γ + δ 𝔼[x]

# average treatment effect of population with x>1/2
# returns estimate of γ + δ 𝔼[x | x>1/2]
cate = np.mean(est.effect(T0_test[X_test>1/2], T1_test[X_test>1/2], X_test[X_test>1/2]))

ValueError: einstein sum subscripts string contains too many subscripts for operand 0

In [65]:
# Estimate expected lift of treatment policy: π(z) = 𝟙{x > 0} over existing policy
Pi0_test = T
Pi1_test = (X_test > 0) * 1.
# returns estimate of γ/2 + δ/√(2π)
policy_effect = np.mean(est.effect(Pi0_test, Pi1_test, X_test))

# Estimate expected lift of treatment policy: π(x) = 𝟙{x > 0} over baseline of no treatment
Pi0_test = np.zeros((X_test.shape[0], n_treatments))
Pi1_test = (X_test > 0) * 1.
# returns estimate of γ/2 + δ/√(2π)
policy_effect = np.mean(est.effect(Pi0_test, Pi1_test, X_test))

ValueError: einstein sum subscripts string contains too many subscripts for operand 0

In [12]:
data = dowhy.datasets.linear_dataset(beta=10,
        num_common_causes=5, 
        num_instruments = 2,
        num_samples=10000,
        treatment_is_binary=True)
df = data["df"]
df.head()

,Z0,Z1,X0,X1,X2,X3,X4,v,y
0,0.0,0.757759,-0.146694,-0.861165,-1.157109,-0.835184,-0.959097,0.0,-6.684585
1,1.0,0.775767,-0.089577,-1.556363,0.287463,-0.794376,0.705783,1.0,6.500541
2,0.0,0.600675,0.702712,-0.902857,0.363468,1.053629,-0.141210,0.0,-0.331293
3,0.0,0.096195,-1.400516,-0.842746,0.893859,1.418935,0.679710,1.0,8.325851
4,0.0,0.307697,0.468576,-3.527561,0.660081,0.328313,-1.151785,1.0,1.585656


In [14]:
from econml.dml import DMLCateEstimator
from sklearn.linear_model import LassoCV

est = DMLCateEstimator(model_y=LassoCV(), model_t=LassoCV())
est.fit(df["y"], df["v"], df["Z0"], df[["X1", "X2"]]) # W -> high-dimensional confounders, X -> features
treatment_effects = est.effect(X_test)

KeyError: "None of [Int64Index([   1,    5,    7,   13,   14,   15,   20,   25,   26,   29,\n            ...\n            9978, 9979, 9980, 9982, 9983, 9986, 9987, 9989, 9994, 9999],\n           dtype='int64', length=5000)] are in the [columns]"